In [1]:
!pip install gspread oauth2client
!pip install google-forms
!pip install -U langchain-openai
!pip install langchain
!pip install peft
!pip install optimum
!pip install auto-gptq

  Preparing metadata (setup.py) ... done
  Created wheel for google-forms: filename=google_forms-0.3-py3-none-any.whl size=1939 sha256=88b72af799c27048e91e4e5c32c6f39db44fb5b8c5c90cb8720fdc1916fee833
  Stored in directory: /root/.cache/pip/wheels/a9/91/e8/f8aa71d1f9fe24ef80c3232a388cb152e3ac3a8164f74b88c7
Successfully built google-forms
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# Path to the downloaded JSON file with your Google Sheets API credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '/content/autograder-88-5a24be97f8a6.json', scope
)

# Authorize the client
client = gspread.authorize(creds)

def download_form_data(spreadsheet_url):
    # Open the spreadsheet
    spreadsheet = client.open_by_url(spreadsheet_url)

    # Get the first (default) sheet
    sheet = spreadsheet.sheet1

    # Get form questions
    questions = sheet.row_values(1)
    print("Form Questions:")
    print(questions)

    # Get form responses
    responses = sheet.get_all_records()
    print("\nForm Responses:")
    for response in responses:
        print(response)

    form_data = {}
    form_data['questions'] = questions
    for question in questions:
        form_data[question] = [response[question] for response in responses]

    return form_data

In [3]:
# URL of the Google Form responses spreadsheet
spreadsheet_url = "https://docs.google.com/spreadsheets/d/14OwVzPSQ9sNyxOcL8CMrF-7y42Xic9x9DJiYWaLIPFc/edit?usp=sharing"

In [4]:
# Call the function to download form data
responses = download_form_data(spreadsheet_url)

Form Questions:
['Timestamp', 'Who is the prime minister of India?', 'Who was the 35th president of the United States of America?']

Form Responses:
{'Timestamp': '31/03/2024 03:03:37', 'Who is the prime minister of India?': 'Narendra Modi', 'Who was the 35th president of the United States of America?': 'John F. Kennedy'}
{'Timestamp': '31/03/2024 03:03:53', 'Who is the prime minister of India?': 'N Modi', 'Who was the 35th president of the United States of America?': 'JFK'}
{'Timestamp': '31/03/2024 03:04:08', 'Who is the prime minister of India?': 'MODI', 'Who was the 35th president of the United States of America?': 'Jack Kennedy'}
{'Timestamp': '31/03/2024 03:19:14', 'Who is the prime minister of India?': 'Atal Bihari Bajpayee', 'Who was the 35th president of the United States of America?': 'Yogi Adityanath'}


In [5]:
responses['questions'][1], responses[(responses['questions'][1])][0]

('Who is the prime minister of India?', 'Narendra Modi')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain import LLMChain
from api_key import newkey

In [ ]:
chat_gpt = ChatOpenAI(openai_api_key=newkey, temperature=0)

In [ ]:
# define prompt template
prompt_template_text = """You are a high school history teacher grading homework assignments. \
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct. \
Grading is binary; therefore, student answers can be correct or wrong. \
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

prompt_temp = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template = prompt_template_text)

In [ ]:
# define output parser function
class OutputParser(BaseOutputParser):
    def parse(self, text: str):
        lower_text = text.lower()
        # return ("wrong" not in lower_text) and ("incorrect" not in lower_text)
        if ("wrong" not in lower_text) and ("incorrect" not in lower_text):
          return 1
        else:
          return 0

In [ ]:
chain = LLMChain(llm=chat_gpt, prompt=prompt_temp, output_parser = OutputParser())

In [ ]:
# Define a question and its correct answer
ques1 = "Who is the prime minister of India?"
correct_answer = responses[ques1][0]

# The model will take the student's answer as input
student_answer = responses[ques1][3]

chain.invoke({'question':ques1, 'correct_answer':correct_answer, 'student_answer':student_answer})['text']

0

In [ ]:
ques_1 = responses['questions'][1]
ans_1 = responses[ques_1][0]
student_answer_list1 = responses[ques1]

print(ques_1, ans_1, student_answer_list1)

Who is the prime minister of India? Narendra Modi ['Narendra Modi', 'N Modi', 'MODI', 'Atal Bihari Bajpayee']


In [ ]:
ques_1 = responses['questions'][1]
ans_1 = responses[ques_1][0]
student_answer_list1 = responses[ques1]

for student in student_answer_list1:
  print(chain.invoke({'question':ques_1, 'correct_answer':ans_1, 'student_answer':student})['text'])

1
1
1
0


In [ ]:
for i in range(1, len(responses['questions'])):
  ques = responses['questions'][i]
  ans = responses[ques][0]
  student_answer_list = responses[ques]
  for student in student_answer_list:
    print(chain.invoke({'question':ques, 'correct_answer':ans, 'student_answer':student})['text'])
    print("\n")

1


1


1


0


1


1


1


0


